## 豆瓣图书爬取
- 武汉理工软件需求工程实验
- 1. 爬取每本图书的书名，作者，评分，引言，详情页链接
- 2. 图书评论爬取，需获取用户名，推荐等级以及具体评论内容，每本书评论数不少于200条

In [65]:
from lxml import etree
import requests
import pandas as pd

# 图书信息获取
url = "https://book.douban.com/top250"

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
    'Cookie':'ll="118254"; bid=WMenNapI6Mg; __gads=ID=a470b7409f2d6b1c-22ad33d5a5d500c7:T=1664008588:RT=1664008588:S=ALNI_MbxjgrTNGWtNVx88Q-s63VQVVNGew; gr_user_id=f94d205f-9d93-4ebb-9ad3-cd7e58530e71; _vwo_uuid_v2=DD841119DB3E9C4DF2A28B04ADDF5F118|70a699be079a06f6705eeee5c3f01cc3; __yadk_uid=HpjVSURmKY70mV0JrEkXR0Jn90VrUBSa; __utmc=30149280; __utmc=81379588; viewed="1007305_5414391"; __gpi=UID=000008c7da735c95:T=1664008588:RT=1666412504:S=ALNI_MbPiWzt-Jw1yySApe-3WItVBbQlOA; ap_v=0,6.0; dbcl2="263836883:1NzNU7Z1ar4"; ck=l_se; _pk_ref.100001.3ac3=%5B%22%22%2C%22%22%2C1666425436%2C%22https%3A%2F%2Faccounts.douban.com%2F%22%5D; _pk_id.100001.3ac3=8f4229e47a43d899.1666365420.3.1666425436.1666419643.; _pk_ses.100001.3ac3=*; __utma=30149280.1501336039.1664008588.1666412421.1666425437.4; __utmz=30149280.1666425437.4.3.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmt_douban=1; __utmb=30149280.1.10.1666425437; __utma=81379588.55727956.1666365420.1666412428.1666425437.3; __utmz=81379588.1666425437.3.3.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmt=1; __utmb=81379588.1.10.1666425437; push_noty_num=0; push_doumail_num=0'
}

# 存储图书信息
total_info = pd.DataFrame(columns=["标题","作者","评分","引言","链接"])

# # 用于存储书名和链接的汇总
# # 以便后续获取评论
# title_list_total = []
# link_list_total = []

for i in range(10):
    start_num = i * 25
    params = {
    'start':start_num
    }
    page_text = requests.get(url=url,headers=headers,params=params).content

    tree = etree.HTML(page_text)
    # 当前页的标题列表
    title_list = tree.xpath('//table//tr[@class="item"]/td[2]/div[@class="pl2"]/a/@title')
    # 当前页的作者列表
    author_list = list(map(lambda x:x.split("/")[0], tree.xpath('//table//tr[@class="item"]/td[2]/p[@class="pl"]/text()')))
    # 当前页的评分列表
    rate_list = tree.xpath('//table//tr[@class="item"]/td[2]//span[@class="rating_nums"]/text()')
    # 当前页的引用列表
    cite_list = [''.join(td.xpath('p[@class="quote"]/span[@class="inq"]/text()')) for td in tree.xpath('//div[@class="indent"]/table//tr[@class="item"]/td[2]')]
    # 对应的链接列表
    link_list = tree.xpath('//table//tr[@class="item"]/td[2]/div[@class="pl2"]/a/@href')

    page_pd = pd.DataFrame({"标题":title_list,"作者":author_list,"评分":rate_list,"引言":cite_list,"链接":link_list})

    total_info = total_info.append(page_pd, ignore_index=True)

total_info.to_csv("2-7豆瓣图书信息.csv",encoding="utf_8_sig")


C:\Windows\Temp\ipykernel_3008\2959218232.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_info = total_info.append(page_pd, ignore_index=True)


In [ ]:
name_list = pd.read_csv("2-7豆瓣图书信息.csv",index_col=0)["标题"].to_list()
link_list = pd.read_csv("2-7豆瓣图书信息.csv",index_col=0)["链接"].to_list()

total_comment = pd.DataFrame(columns=["评论人","推荐等级","具体内容","书名"])

for i in range(len(name_list)):
    # 书的标题
    name_item = name_list[i]
    # 书的链接
    link_item = link_list[i]
    link_item = link_item + 'comments/'
    # 读取200条评论的循环
    comment_people_total = []
    comment_rate_total = []
    comment_content_total = []

    for ii in range(10):
        params_content = {
        'start':ii * 20,
        'limit':(ii+1) * 20,
        }
        onebook_text = requests.get(url=link_item, headers=headers,params=params_content).text
        tree_comment = etree.HTML(onebook_text)
        # 评论人的姓名
        comment_people = tree_comment.xpath('//div[@class="comment"]/h3/span[@class="comment-info"]/a[1]/text()')
        # 推荐等级
        # comment_rate = tree_comment.xpath('//li[@class="comment-item"]/div[@class="comment"]/h3/span[@class="comment-info"]//@title')
        comment_rate = [''.join(li.xpath('div[@class="comment"]/h3/span[@class="comment-info"]//@title')) \
            for li in tree_comment.xpath('//div[@class="comment-list score "]/ul/li[@class="comment-item"]')]
        # 评论内容
        # comment_content = tree_comment.xpath('//div[@class="comment"]/p[@class="comment-content"]/span[@class="short"]/text()')
        comment_content = [''.join(li.xpath('div[@class="comment"]/p[@class="comment-content"]/span[@class="short"]/text()')) \
            for li in tree_comment.xpath('//div[@class="comment-list score "]/ul/li[@class="comment-item"]')]


        # 存到total里面
        comment_people_total.extend(comment_people)
        comment_rate_total.extend(comment_rate)
        comment_content_total.extend(comment_content)
       
    page_pd = pd.DataFrame({"评论人":comment_people_total,"推荐等级":comment_rate_total,"具体内容":comment_content_total})
    page_pd["书名"] = name_item
    
    total_comment = total_comment.append(page_pd, ignore_index=True)

total_comment.to_csv("2-7豆瓣图书评论.csv",encoding="utf_8_sig")


In [2]:
from lxml import etree
import requests
import pandas as pd

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
    'Cookie':'ll="118254"; bid=WMenNapI6Mg; __gads=ID=a470b7409f2d6b1c-22ad33d5a5d500c7:T=1664008588:RT=1664008588:S=ALNI_MbxjgrTNGWtNVx88Q-s63VQVVNGew; gr_user_id=f94d205f-9d93-4ebb-9ad3-cd7e58530e71; _vwo_uuid_v2=DD841119DB3E9C4DF2A28B04ADDF5F118|70a699be079a06f6705eeee5c3f01cc3; __yadk_uid=HpjVSURmKY70mV0JrEkXR0Jn90VrUBSa; __utmc=30149280; __utmc=81379588; viewed="1007305_5414391"; __gpi=UID=000008c7da735c95:T=1664008588:RT=1666412504:S=ALNI_MbPiWzt-Jw1yySApe-3WItVBbQlOA; ap_v=0,6.0; dbcl2="263836883:1NzNU7Z1ar4"; ck=l_se; _pk_ref.100001.3ac3=%5B%22%22%2C%22%22%2C1666425436%2C%22https%3A%2F%2Faccounts.douban.com%2F%22%5D; _pk_id.100001.3ac3=8f4229e47a43d899.1666365420.3.1666425436.1666419643.; _pk_ses.100001.3ac3=*; __utma=30149280.1501336039.1664008588.1666412421.1666425437.4; __utmz=30149280.1666425437.4.3.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmt_douban=1; __utmb=30149280.1.10.1666425437; __utma=81379588.55727956.1666365420.1666412428.1666425437.3; __utmz=81379588.1666425437.3.3.utmcsr=accounts.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; __utmt=1; __utmb=81379588.1.10.1666425437; push_noty_num=0; push_doumail_num=0'
}

In [ ]:
from multiprocessing.dummy import Pool
name_list = pd.read_csv("2-7豆瓣图书信息.csv",index_col=0)["标题"].to_list()
link_list = pd.read_csv("2-7豆瓣图书信息.csv",index_col=0)["链接"].to_list()
name_urls = []
for i in range(len(name_list)):
    dic_item = {
        "name": name_list[i],
        "link":link_list[i]
    }
    name_urls.append(dic_item)


total_comment = pd.DataFrame(columns=["评论人","推荐等级","具体内容","书名"])

def onebook_download(name_url):
    global total_comment
    name_item = name_url['name']
    link_item = name_url['link'] + 'comments/'
    # 读取200条评论的循环
    comment_people_total = []
    comment_rate_total = []
    comment_content_total = []

    for ii in range(10):
        params_content = {
        'start':ii * 20,
        'limit':(ii+1) * 20,
        }
        onebook_text = requests.get(url=link_item, headers=headers,params=params_content).text
        tree_comment = etree.HTML(onebook_text)
        # 评论人的姓名
        comment_people = tree_comment.xpath('//div[@class="comment"]/h3/span[@class="comment-info"]/a[1]/text()')
        # 推荐等级
        comment_rate = [''.join(li.xpath('div[@class="comment"]/h3/span[@class="comment-info"]//@title')) \
            for li in tree_comment.xpath('//div[@class="comment-list score "]/ul/li[@class="comment-item"]')]
        # 评论内容
        comment_content = [''.join(li.xpath('div[@class="comment"]/p[@class="comment-content"]/span[@class="short"]/text()')) \
            for li in tree_comment.xpath('//div[@class="comment-list score "]/ul/li[@class="comment-item"]')]
        # print(comment_people)
        # 存到total里面
        comment_people_total.append(comment_people)
        comment_rate_total.append(comment_rate)
        comment_content_total.append(comment_content)
    page_pd = pd.DataFrame({"评论人":comment_people_total,"推荐等级":comment_rate_total,"具体内容":comment_content_total})
    page_pd["书名"] = name_item
    total_comment = total_comment.append(page_pd, ignore_index=True)

pool = Pool(4)
pool.map(onebook_download, name_urls)

pool.close()
pool.join()

total_comment.to_csv("2-7豆瓣图书评论.csv",encoding="utf_8_sig")

